In [102]:
import boto3
import time
import traceback
import json

In [103]:
VERBOSE = 1

In [63]:
def get_client(r_name, r_type):
    client = boto3.client(r_type, region_name=r_name)
    return client

In [64]:
def get_regions(client):
  """ Build a region list """

  reg_list = []
  regions = client.describe_regions()
  data_str = json.dumps(regions)
  resp = json.loads(data_str)
  region_str = json.dumps(resp['Regions'])
  region = json.loads(region_str)
  for reg in region:
    reg_list.append(reg['RegionName'])
  return reg_list

In [65]:
def delete_ec2_instances(c_session):
    print("Starting delete_ec2_instances")
    try:
        all_instances = c_session.describe_instances()
        for reservations in all_instances['Reservations']:
            for instance in reservations['Instances']:
                instance_id = instance['InstanceId']
                print(f"Terminating instance - {instance_id}")
                c_session.terminate_instances(InstanceIds=[instance_id])
                print(f"Terminated instance - {instance_id}")
    except:
        strError = traceback.format_exc()
        
        print(strError)
    finally:
        print("Stopping delete_ec2_instances")

In [66]:
def delete_ec2_amis(c_session):
    print("Starting delete_ec2_amis")
    try:
        aws_resources = c_session.describe_images(Owners=["357841589350"])
        for aws_resource in aws_resources['Images']:
            aws_resource_id = aws_resource['ImageId']
            print(f"Deleting AWS Resource: {aws_resource_id}")
            c_session.deregister_image(ImageId=aws_resource_id)
            print(f"AWS Resource deleted - {aws_resource_id}")
    except:
        print(traceback.format_exc())
    print("Stopping delete_ec2_amis")

In [67]:
def delete_ec2_snapshots(c_session):
    print("Starting delete_ec2_snapshots")
    try:
        aws_resources = c_session.describe_snapshots(OwnerIds=["357841589350"],MaxResults=10,)
        for aws_resource in aws_resources['Snapshots']:
            snapshot_id = aws_resource['SnapshotId']
            print(f"Deleting AWS Resource: {snapshot_id}")
            c_session.delete_snapshot(SnapshotId=snapshot_id)
            print(f"Resource deleted - {snapshot_id}")    
    except:
        print(traceback.format_exc())
    print("Stopping delete_ec2_snapshots")

In [68]:
def delete_elbv2(c_session):
    print("Starting delete_elbv2")
    try:
        aws_resources = c_session.describe_load_balancers()
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            print(f"Deleting aws resource: {aws_resource['LoadBalancerArn']}")
            c_session.delete_load_balancer(LoadBalancerArn=aws_resource['LoadBalancerArn'])
            print(f"Deleted aws resource: {aws_resource['LoadBalancerArn']}")
    except:
        print(traceback.format_exc())
    print("Stopping delete_elbv2")

In [69]:
def delete_nat_gateways(c_session):
    print("Starting delete_nat_gateways")
    try:
        aws_resources = c_session.describe_nat_gateways()
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            aws_resource_id = aws_resource['NatGatewayId']
            print(f"Deleting aws resource: {aws_resource_id}")
            c_session.delete_nat_gateway(NatGatewayId=aws_resource_id)
            print(f"AWS Resource deleted - {aws_resource_id}")
    except:
        print(traceback.format_exc())
    print("Stopping delete_nat_gateways")

In [70]:
def delete_elastic_ips(c_session):
    print("Starting delete_elastic_ips")
    try:
        aws_resources = c_session.describe_addresses()
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            print(aws_resource)
            print(f"Deleting aws resource")
            if 'AssociationId' in aws_resource.keys():
                c_session.disassociate_address(AssociationId=aws_resource['AssociationId'])
            c_session.release_address(AllocationId=aws_resource['AllocationId'])
            print(f"AWS Resource deleted - {aws_resource['AllocationId']}")
    except:
        print(traceback.format_exc())

    print("Stopping delete_elastic_ips")

In [71]:
def delete_asg(c_session):
    print("Starting delete_asg")
    try:
        aws_resources = c_session.describe_addresses()
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            print(aws_resource)
            print(f"Deleting aws resource")
            c_session.delete_auto_scaling_group(AutoScalingGroupName=aws_resource['AutoScalingGroupName'], ForceDelete=True)
            print(f"AWS Resource deleted - {aws_resource['AutoScalingGroupName']}")
    except:
        print(traceback.format_exc())

    print("Stopping delete_asg")

In [95]:
def get_default_vpcs(client):
  vpc_list = []
  vpcs = client.describe_vpcs(
    Filters=[
      {
          'Name' : 'isDefault',
          'Values' : [
            'false',
          ],
      },
    ]
  )
  vpcs_str = json.dumps(vpcs)
  resp = json.loads(vpcs_str)
  data = json.dumps(resp['Vpcs'])
  vpcs = json.loads(data)
  
  for vpc in vpcs:
    vpc_list.append(vpc['VpcId'])  
  
  return vpc_list

In [96]:
c_session = get_client("ap-southeast-2", 'ec2')

get_default_vpcs(c_session)

['vpc-06189fa2241b93c7c',
 'vpc-01904c3aada02eb95',
 'vpc-03fd2ec8c64b656ce',
 'vpc-029bacde59f96ad2e']

In [73]:
def del_igw(ec2, vpcid):
  """ Detach and delete the internet-gateway """
  vpc_resource = ec2.Vpc(vpcid)
  igws = vpc_resource.internet_gateways.all()
  if igws:
    for igw in igws:
      try:
        print("Detaching and Removing igw-id: ", igw.id) if (VERBOSE == 1) else ""
        igw.detach_from_vpc(
          VpcId=vpcid
        )
        igw.delete(
          # DryRun=True
        )
      except boto3.exceptions.Boto3Error as e:
        print(e)

In [74]:
def del_sub(ec2, vpcid):
  """ Delete the subnets """
  vpc_resource = ec2.Vpc(vpcid)
  subnets = vpc_resource.subnets.all()
  default_subnets = [ec2.Subnet(subnet.id) for subnet in subnets if subnet.default_for_az]
  
  if default_subnets:
    try:
      for sub in default_subnets: 
        print("Removing sub-id: ", sub.id) if (VERBOSE == 1) else ""
        sub.delete(
          # DryRun=True
        )
    except boto3.exceptions.Boto3Error as e:
      print(e)

In [75]:
def del_rtb(ec2, vpcid):
  """ Delete the route-tables """
  vpc_resource = ec2.Vpc(vpcid)
  rtbs = vpc_resource.route_tables.all()
  if rtbs:
    try:
      for rtb in rtbs:
        assoc_attr = [rtb.associations_attribute for rtb in rtbs]
        if [rtb_ass[0]['RouteTableId'] for rtb_ass in assoc_attr if rtb_ass[0]['Main'] == True]:
          print(rtb.id + " is the main route table, continue...")
          continue
        print("Removing rtb-id: ", rtb.id) if (VERBOSE == 1) else ""
        table = ec2.RouteTable(rtb.id)
        table.delete(
          # DryRun=True
        )
    except boto3.exceptions.Boto3Error as e:
      print(e)

In [76]:
def del_acl(ec2, vpcid):
  """ Delete the network-access-lists """
  
  vpc_resource = ec2.Vpc(vpcid)      
  acls = vpc_resource.network_acls.all()

  if acls:
    try:
      for acl in acls: 
        if acl.is_default:
          print(acl.id + " is the default NACL, continue...")
          continue
        print("Removing acl-id: ", acl.id) if (VERBOSE == 1) else ""
        acl.delete(
          # DryRun=True
        )
    except boto3.exceptions.Boto3Error as e:
      print(e)

In [77]:
def del_sgp(ec2, vpcid):
  """ Delete any security-groups """
  vpc_resource = ec2.Vpc(vpcid)
  sgps = vpc_resource.security_groups.all()
  if sgps:
    try:
      for sg in sgps: 
        if sg.group_name == 'default':
          print(sg.id + " is the default security group, continue...")
          continue
        print("Removing sg-id: ", sg.id) if (VERBOSE == 1) else ""
        sg.delete(
          # DryRun=True
        )
    except boto3.exceptions.Boto3Error as e:
      print(e)


In [105]:
def del_vpc(ec2, vpcid):
  """ Delete the VPC """
  vpc_resource = ec2.Vpc(vpcid)
  try:
    print("Removing vpc-id: ", vpc_resource.id)
    vpc_resource.delete(
      # DryRun=True
    )
  except boto3.exceptions.Boto3Error as e:
      print(e)
      print("Please remove dependencies and delete VPC manually.")
  except:
      print(traceback.format_exc())
  #finally:
  #  return status

In [141]:
def detach_vpn_gateways(c_session):
    print("Starting detach_vpn_gateways")
    aws_resources = c_session.describe_vpn_gateways()
    for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
        try:
            for vpn_a in aws_resource['VpcAttachments']:
                if vpn_a['State'] != "detached":
                    print(f"AWS Resource Detaching - {vpn_a}")
                    c_session.detach_vpn_gateway(VpnGatewayId=aws_resource['VpnGatewayId'], VpcId = vpn_a['VpcId'])
                    print(f"AWS Resource Detached - {aws_resource['VpnGatewayId']}, {vpn_a['VpcId']}")
        except:
            print(traceback.format_exc())
    print("Stopping detach_vpn_gateways")

In [145]:
def delete_vpn_gateways(c_session):
    print("Starting delete_vpn_gateways")
    aws_resources = c_session.describe_vpn_gateways()
    for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
        try:
            print(f"Deleting aws resource - {aws_resource}")
            c_session.delete_vpn_gateway(VpnGatewayId=aws_resource['VpnGatewayId'])
            print(f"AWS Resource deleted - {aws_resource['VpnGatewayId']}")
        except:
            print(traceback.format_exc())

    print("Starting delete_vpn_gateways")

In [174]:
def delete_vpcs(ec2):
    print("Starting delete_vpcs")
    try:
        aws_resources = c_session.describe_vpcs()
        Filters=[
          {
                  'Name' : 'isDefault',
                  'Values' : [
                    'false',
                  ],
              },
            ]
        
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            try:
                #print(f"aws resource - {aws_resource}")
                if aws_resource['IsDefault'] == False:
                    vpc_id = aws_resource['VpcId']
                    print(f"Deleting aws resource")
                    del_igw(ec2, vpc_id)
                    del_sub(ec2, vpc_id)
                    del_rtb(ec2, vpc_id)
                    del_acl(ec2, vpc_id)
                    del_sgp(ec2, vpc_id)
                    del_vpc(ec2, vpc_id)
                    print(f"AWS Resource deleted - {aws_resource['VpcId']}")
            except:
                print(traceback.format_exc())
    except:
        print(traceback.format_exc())

    print("Stopping delete_vpcs")

In [ ]:
def delete_vpn_connections(c_session):
    print("Starting delete_vpn_connections")
    try:
        aws_resources = c_session.describe_vpn_connections()
        for aws_resource in aws_resources[list(aws_resources.keys())[0]]:
            print(aws_resource)
            vpn_conn_id = aws_resource['VpnConnectionId']
            print(f"Deleting aws resource - {vpn_conn_id}")
            
            c_session.delete_vpn_connection(VpnConnectionId=vpn_conn_id)
            print(f"AWS Resource deleted - {vpn_conn_id}")
    except:
        print(traceback.format_exc())

    print("Stopping delete_elastic_ips")

In [58]:
ec2_client = boto3.client('ec2')

In [56]:
all_regions = get_regions(ec2_client)
print(all_regions)

['ap-south-1', 'eu-north-1', 'eu-west-3', 'eu-west-2', 'eu-west-1', 'ap-northeast-3', 'ap-northeast-2', 'ap-northeast-1', 'ca-central-1', 'sa-east-1', 'ap-southeast-1', 'ap-southeast-2', 'eu-central-1', 'us-east-1', 'us-east-2', 'us-west-1', 'us-west-2']


In [173]:
for r_name in all_regions:
    try:
        #time.sleep(1)
        print(f"----------------------Working with region: {r_name}")
        c_session = get_client(r_name, 'ec2')
        #delete_asg(c_session)
        c_session = get_client(r_name, 'ec2')
        #delete_ec2_instances(c_session)
        #delete_ec2_amis(c_session)
        #delete_ec2_snapshots(c_session)
        c_session = get_client(r_name, 'elbv2')
        #delete_elbv2(c_session)
        c_session = get_client(r_name, 'ec2')
        #delete_nat_gateways(c_session)
        #delete_elastic_ips(c_session)
        delete_vpn_connections(c_session)
        detach_vpn_gateways(c_session)
        delete_vpcs(c_session)
        
        ec2 = boto3.resource('ec2', region_name = r_name)
        delete_vpcs(ec2)
    except:
        print(f"Error while working with region {r_name}")

----------------------Working with region: ap-south-1
Starting delete_vpn_connections
{'CustomerGatewayId': 'cgw-0a70aa362980e0328', 'Category': 'VPN', 'State': 'deleted', 'Type': 'ipsec.1', 'VpnConnectionId': 'vpn-0bed813d478993469', 'VpnGatewayId': 'vgw-0e36a090a78beef52', 'GatewayAssociationState': 'associated', 'Options': {'EnableAcceleration': False, 'StaticRoutesOnly': False}, 'Routes': []}
Deleting aws resource - vpn-0bed813d478993469
Traceback (most recent call last):
  File "C:\Users\atingupta2005\AppData\Local\Temp\ipykernel_1996\2112798752.py", line 10, in delete_vpn_connections
    c_session.delete_vpn_connection(VpnConnectionId=vpn_conn_id)
  File "D:\23-trainings\2-Confirmed\Jan-24-AWS-Labs\GH\cleanup\venv\lib\site-packages\botocore\client.py", line 553, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "D:\23-trainings\2-Confirmed\Jan-24-AWS-Labs\GH\cleanup\venv\lib\site-packages\botocore\client.py", line 1009, in _make_api_call
    raise error_c

In [149]:
c_session = get_client("ap-south-1", 'ec2')

In [151]:
aws_resources = c_session.describe_vpn_gateways()

In [152]:
aws_resources

{'VpnGateways': [{'State': 'available',
   'Type': 'ipsec.1',
   'VpcAttachments': [{'State': 'detached', 'VpcId': 'vpc-0ddaefd6ed24b1d05'}],
   'VpnGatewayId': 'vgw-0e36a090a78beef52',
   'AmazonSideAsn': 64512,
   'Tags': [{'Key': 'Name', 'Value': 'gateway_demo_aj'}]}],
 'ResponseMetadata': {'RequestId': 'a5d8f11e-e41c-43e7-86e2-0927b635464f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a5d8f11e-e41c-43e7-86e2-0927b635464f',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '845',
   'date': 'Wed, 24 Jan 2024 06:10:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [155]:
aws_resources = c_session.describe_vpn_connections()

In [169]:
aws_resources['VpnConnections'][0]['VpnConnectionId']

'vpn-0bed813d478993469'

In [168]:
c_session.delete_vpn_connection(VpnConnectionId='vpn-0bed813d478993469')

ClientError: An error occurred (InvalidVpnConnectionID.NotFound) when calling the DeleteVpnConnection operation: The vpnConnection ID 'vpn-0bed813d478993469' does not exist

In [170]:
c_session.delete_vpn_gateway(VpnGatewayId='vgw-0e36a090a78beef52')

{'ResponseMetadata': {'RequestId': '7be3759b-9cf5-46a4-8602-a78ad35538d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7be3759b-9cf5-46a4-8602-a78ad35538d8',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '233',
   'date': 'Wed, 24 Jan 2024 06:18:42 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}